# PytzMLS2018:  Lab 3: Introduction to deep learning

<center>**Anthony Faustine (sambaiga@gmail.com)**</center>



## Import all necessary libries and modules we will use

In [ ]:
import sys
sys.path.append('../src')
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
from ploting import *

%matplotlib inline

## Part 1. Feed-forward Neural Network

**Learning goal**: How to implement a feed-forward neural network by using PyTorch. In this lab we will train a feed-forward neural network by using PyTorch. We will do the following steps in order:

**Task**: Build MLP classifier to recognize handwritten digits using the MNIST dataset. MNIST contains 70,000 images: 60,000 for training and 10,000 for testing. The images are grayscale, 28x28 pixels, and centered to reduce preprocessing and get started quicker. 

** Procedure**

1. Load the training and test datasets using DataLoader
2. Define a Feedforwad Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

### 1.1 Load dataset

We will use DataLoader and TensorDataset (from torch.utils.data) for convinience in data handling. You can create your custom dataset class by inheriting Dataset with some required member functions.

In [ ]:
train_dataset = datasets.MNIST(root='../data/mnist', 
                            train=True, # this is training data
                            transform=transforms.ToTensor(),# Converts a PIL.Image or numpy.ndarray to
                                                           # torch.FloatTensor of shape (C x H x W) and normalize in the range [0.0, 1.0]
                            download=True)

test_dataset = datasets.MNIST(root='../data/mnist', 
                           train=False, 
                           transform=transforms.ToTensor())

### 1.1.1 Make iteratable data loader

In [ ]:
batch_size = 32


train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

### 1.1.2  Visualize train dataset


Let's inspect a few examples. The MNIST dataset contains only grayscale images. For more advanced datasets, we'll have the three color channels (RGB).

In [ ]:
beatify(fig_width=6)
fig, axs = plt.subplots(2,5, facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.25, wspace=.1)
axs = axs.ravel()

for i in range(10):
    axs[i].imshow(train_dataset.train_data[i].numpy(), cmap='gray', interpolation='none')
    axs[i].set_title('%i' % train_dataset.train_labels[i], fontsize=12)
    plt.tight_layout()
    axs[i].set_xticks([])
    axs[i].set_yticks([])
  
  

### 1.2. Define  MLP model

In this step we define MLP model. For simplicity, we will use 3-layer MLP in which each layer is a fully-connected layer and apply RELU activation for each layer with exception to the output layer. 

**Questions**
1. Why can't we apply RelU activation function to the output layer?
2. Which output activation function is appropriate for this type of problem?

In [ ]:
def MLP(input_dim, hidden_dim, output_dim):
    
    
    model = nn.Sequential(
    nn.Linear(input_dim,hidden_dim), 
    nn.ReLU(),
    nn.Linear(hidden_dim,hidden_dim), 
    torch.nn.ReLU(),
    nn.Linear(hidden_dim,output_dim)
        
    )
    
    return model


In [ ]:
input_dim = 784 #28X28
hidden_dim = 16
output_dim = 10


model = MLP(input_dim, hidden_dim, output_dim)
print(model)

### 1.3. Define a Loss function and Optimizer¶

Since this is multi-class classification we will use **Cross Entropy loss**. PyTorch provide **Cross Entropy loss**  loss function which combines a softmax layer and the **Cross Entropy loss** together and it is more numerically stable than using them separately. Thus it is why we didnt  apply softmax activation after the output layer while defining MLP model. See the last model definition above.

We will use SGD with momentum as our optimizer. When we create an optimizer in PyTorch, we need to pass parameters that we want to optimize (train) as input arguments. We can retrieve all trainable parameters of the model by calling **model.parameters()**.

**Questions**
1. Write an expression of a softmax function.
2. What are the advantage of using SGD with momentum in training neural network?

In [ ]:
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer =  torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
device = torch.device("cuda:{0}".format(args.cuda_num) if torch.cuda.is_available() else "cpu")

### 1.4. Train the network
Now, we need to train the model. For each full coverage of train dataset, we just need to do a forward pass computation with a mini-batch of dataset and a backward pass to compute gradients followed by a step of optimization. We need to do this for a reasonable number of iterations.

In [ ]:
def train(model, optimizer, criterion, num_epochs, device):
    
    
    total_loss = []
    print("Start training")
    model=model.to(device)
    criterion=criterion.to(device)
    
    for epoch in range(num_epochs):
        
        training_loss = []
        
        model.train()
        
        for i, (images, labels) in enumerate(train_loader):
            
            images = images.view(-1, 28*28).to(device)
            labels = labels
        
            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()
        
            # Forward pass to get output/logits
            outputs = model(images)
        
            # Calculate Loss: softmax --> cross entropy loss
            loss = criterion(outputs, labels)
        
            # Getting gradients w.r.t. parameters
            loss.backward()
            
            
            training_loss.append(loss.item())
        
            # Updating parameters
            optimizer.step()
        
            # print statistics
            # print statistics
           
            if i %4000 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.4f}'.format(
                epoch+1, i * len(images), len(train_loader.dataset),
                100. * i / len(train_loader), np.mean(training_loss)))
             
              
        total_loss.append(np.mean(training_loss))     
        
       
    return total_loss    
          

In [ ]:
#train models
total_loss = train(model,optimizer, criterion, 10, device)

### 1.4.1 Visualize training loss

In [ ]:
beatify(fig_width=4)
plt.plot(total_loss, label="rate={}".format(learning_rate))
plt.title("Training Loss")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend();

### 1.5 Test the network on the test data

In [ ]:
def test(model, device):
    
    model=model.to(device)
    model.eval()
    
    correct = 0
    predictions=[]
    ground_t=[]
    
    for i, (images, labels) in enumerate(test_loader):
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)

        outputs = model(images)
        
        # Get predictions from the maximum value
        _, pred = torch.max(outputs.data, 1)
        predictions  += pred.numpy().tolist()
        ground_t+= labels.data.numpy().tolist()
        #correct += pred.eq(labels.data.view_as(pred)).cpu().sum()
        correct += torch.sum(pred == labels.data)
    
    print('')
    predictions = np.array(predictions)
    ground_t    = np.array(ground_t)   
    accuracy = 100. * correct / len(test_loader.dataset)
        
    print("Accuracy: {}%".format(accuracy))
    
    return ground_t, predictions
    
    

In [ ]:
ground_t, predictions = test(model, device)

### 1.5.1 VIsualize Results

In [ ]:
correct_indices = np.nonzero(predictions == ground_t)[0]
incorrect_indices = np.nonzero(predictions != ground_t)[0]
print()
print(len(correct_indices)," classified correctly")
print(len(incorrect_indices)," classified incorrectly")

### 1.5.1.1 Visualize correct prediction

In [ ]:
beatify(fig_width=8)
indices = np.random.permutation(correct_indices)
figure_evaluation = plt.figure()

# plot 9 correct predictions
for i, correct in enumerate(indices[:9]):
    plt.subplot(6,3,i+1)
    plt.imshow(test_dataset.test_data[correct].numpy(),  cmap='gray', interpolation='none')
    plt.title("Predicted {}, Actual {}".format(predictions[correct], ground_t[correct]))
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

#### 1.5.1.2 Visualize incorrect prediction

In [ ]:
beatify(fig_width=8)
indices = np.random.permutation(incorrect_indices)
figure_evaluation = plt.figure()

# plot 9 correct predictions
for i, correct in enumerate(indices[:9]):
    plt.subplot(6,3,i+1)
    plt.imshow(test_dataset.test_data[correct].numpy(), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Actual {}".format(predictions[correct], ground_t[correct]))
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

### 1.5.2 Experiment with other classification metrics implemented in sklearn

**Confusion matrix**

A confusion matrix is a technique for summarizing the performance of a classification algorithm. It is a summary of prediction results on a classification problem. The number of correct and incorrect predictions are summarized with count values and broken down by each class. This is the key to the confusion matrix.

Classification accuracy alone can be misleading if you have an unequal number of observations in each class or if you have more than two classes in your dataset.

Calculating a confusion matrix can give you a better idea of what your classification model is getting right and what types of errors it is making.

For detail on confusion matrix refer [here](https://machinelearningmastery.com/confusion-matrix-machine-learning/) and [here](http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/)

We use [sklearn library](http://scikit-learn.org/stable/) to calculate confusion matrix and [searborn library](https://seaborn.pydata.org/) to plot the confusion matrix.

In [ ]:
from sklearn.metrics import  confusion_matrix
cm=confusion_matrix(ground_t, predictions)
cm

Our class in digits from 0 to 10.

In [ ]:
class_names = np.unique(ground_t)
class_names

In [ ]:
import seaborn as sn

def confusion_matrix_plot(cm, class_names):
    df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
    try:
        heatmap = sn.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
        
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right')
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=0, ha='right')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
     

In [ ]:
beatify(fig_width=4)
confusion_matrix_plot(cm, class_names)

### 1.5.3 [** Recall, Precision and F-measure**](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/)

**Recall (Sensitivity)**: is the ratio of correctly predicted positive observations to the all observations in actual class. 
$$Recall = \frac{TP}{TP + FN}$$

**Precision**  is the ratio of correctly predicted positive observations to the total predicted positive observations. 

$$Precision = \frac{TP}{TP + FP}$$

**F1 score**: is the weighted average of Precision and Recall, it takes both false positives and false negatives into account.

$$F1 Score = \frac{2\cdot Recall \cdot Precision} {Recall + Precision}$$

In [ ]:
from sklearn.metrics import f1_score, precision_score,recall_score

In [ ]:
recal = recall_score(ground_t, predictions, average='weighted')
prec  = precision_score(ground_t, predictions, average='weighted')
f1    = f1_score(ground_t, predictions, average='weighted')
print("Recall: {0:.4f}, Precision: {1:.4f}, F-score: {2:.4f}".format(recal,prec, f1))

## Part 2. Weight Initilization 

Before  training neural network we have to initialize its parameters. By default pytorch itinilize this parameters for you. However you can use different techniques such as gausian random initilization, kaiming and xavier initialization. In this part we will experiment with the following initilization zero, kaiming and xavier initilization and comprae their results with defaulty initilization. We will modify the previous model by adding weight initilization.

In pytorch most initilization functions are implemented in [**torch.nn.init**](http://pytorch.org/docs/0.3.1/nn.html#torch-nn-init). As discussed in tutorial we need to define initilization function that we apply to model parameters.

### 2.1. Zero weight initilization

In this case we initilize the weight to zero by first defining a function that willinitilize  weight of the nn.Linear module as follows.

In [ ]:
def zero_weights_init(m): 
    if isinstance(m, nn.Linear):
        m.weight.data.fill_(0.0)
        m.bias.data.fill_(0.0)

Define a model and apply zero weight initilization

In [ ]:
model_weight_zero =  MLP(input_dim, hidden_dim, output_dim)

Apply the weight init function to the defined model

In [ ]:
model_weight_zero.apply(zero_weights_init)
print(model_weight_zero)

 Let us check to see if the initial weight is zero

In [ ]:
for  param in model_weight_zero.parameters():
    print(param.data[0])

 We can now build and test our model

In [ ]:
optimizer =  torch.optim.SGD(model_weight_zero.parameters(), lr=learning_rate, momentum=0.9)
total_loss_zero_weight = train(model_weight_zero, optimizer, criterion, 10, device)

Let us compare the training loss when zero initilization is applied

In [ ]:
plt.plot(total_loss, label=" default weight")
plt.plot(total_loss_zero_weight, label=" zero weight")
plt.title("Training Loss")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()

#### Questions
1. What can you conclude on the above results?

### 2.2 Xavier weight initilization

Implemented in [**torch.nn.init.xavier_uniform**](http://pytorch.org/docs/0.3.1/nn.html#torch-nn-init). We will repeat the same procedure as in 2.1 but with xavier weight initilizer.

In [ ]:
def xavier_weights_init(m): 
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0.0)
        

        
model_xavier =  MLP(input_dim, hidden_dim, output_dim)

model_xavier.apply(xavier_weights_init)
print(model_xavier)        
        

In [ ]:
for  param in model_xavier.parameters():
    print(param.data[0])

In [ ]:
optimizer =  torch.optim.SGD(model_xavier.parameters(), lr=learning_rate, momentum=0.9)
total_loss_xavier = train(model_xavier, optimizer, criterion, 10, device)

### 2.3 Kaiming weight initilization

Implemented in [**torch.nn.init.kaiming_normal()**](http://pytorch.org/docs/0.3.1/nn.html#torch-nn-init). We will repeat the same procedure as in 2.2 but with kaiming weight initilizer.

In [ ]:
def kaiming_weights_init(m): 
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0)

In [ ]:
model_kaiming =  MLP(input_dim, hidden_dim, output_dim)

model_kaiming.apply(kaiming_weights_init)
print(model_kaiming)

In [ ]:
for param in model_kaiming.parameters():
    print(param.data[0])

In [ ]:
optimizer =  torch.optim.SGD(model_kaiming.parameters(), lr=learning_rate, momentum=0.9)

In [ ]:
total_loss_kaiming = train(model_kaiming, optimizer, criterion, 10, device)

Now let compare the result of the three initilization; default, Xavier and kaiming.

**Task**: Plot the the three plot on the same graph

In [ ]:
plt.plot(total_loss, label="default ")
plt.plot(total_loss_xavier, label=" xavier")
plt.plot(total_loss_kaiming, label=" kaiming ")
plt.title("Training Loss")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()

**Questions**:
1. Which initilization techniques is the best?

## Task: 
Use the best weight initilization to experiments with different learning rate ($0.0001,0.001, 0.01,1$) and different optimisation algorithm such as Adam (`torch.optim.Adam()`) 

### 2.3 Compare test results
Compare accuracy of the four model training with different initilization technoique

In [ ]:
ground_zero, prediction_zero=test(model_weight_zero)

In [ ]:
ground_xavier, prediction_xavier=test(model_xavier)

In [ ]:
ground_kaming, prediction_kaiming=test(model_kaiming)

**Questions**

1. What can you conclude on weight initilization ?

## Part 3 Model complexity, overfitting & regularization

**Overfitting** occurs when improving the model's training loss (its performance on training data) comes at the expense of its generalisation ability (its performance on unseen test data). Generally it is a symptom of the model complexity increasing to fit the peculiarities (outliers) of the training data too accurately, causing it not to generalize well to new unseen (test) data. Overfitting is usually indicated when:

- training & validation loss starts decreasing at different rates,
- validation error starts increasing while training error still goes down,
- training error reaches 0.

**Underfitting** is the opposite: when a model cannot fit the training data well enough (usually a sign to train for longer or add more parameters to the model).

We can increase model complexity by adding more layers (i.e. more parameters). We can control or reduce the model complexity of an architecture using a family of techniques called **regularisers**. 

**Questions**

1. Which regularization techniques can be used to control model complexity?
2. What is the difference between $L1$ and $L2$ regularizers?



### 3.1 Early stopping

Neural networks are nonlinear models and can have very complicated optimization landscapes. Stochastic gradient based methods for optimizing these loss functions do not proceed monotonically (i.e. does not just keep going up). Sometimes the loss can go down for a while before it goes up to reach a better part of parameter space later. How do we know when to stop training?


**Early stopping** is one technique that helps with this. It is added to the training routine and means that we periodically evaluate the model's performance on the validation set. If the performance on the validation set starts becoming worse we know we have reached the point of overfitting (usually), so it usually makes sense to stop training and not waste any more computations.

There are different ways of implementing this early stopping in practise:

1. **Basic early stopping**:stop training as soon as the model starts doing worse on validation data.
2. **Early stopping with patience**: don't stop training immediately once validation accuracy degrades, but wait for P more epochs, and reset P if the model starts improving again within this timeframe.
3. **Eaarly stopping after T epoch**: training for T epochs, and simply selecting the best model based on validation score over the entire T epochs.


**Questions**

1. Why does early stopping rely on the model performance on validation set and not training set ?
2. What are the pros and cons of these different early stopping methods?


###  Basic early stopping

First let write a function to compare train and validation performance.

In [ ]:
def fit_model(model, optimizer, criterion, num_epochs, device):
    
    total_train_loss = []
    total_val_loss = []
    total_train_acc = []
    total_val_acc = []
    
    model.to(device)
    criterion.to(device)
    
    print("Start training")
    for epoch in range(num_epochs):
        
        #Train
        running_loss = 0
        correct  = 0
        model.train()
        for i, (images, labels) in enumerate(train_loader):
            images = images.view(-1, 28*28).to(device)
            labels = labels.to(device)
        
            optimizer.zero_grad()
            outputs = model(images)
        
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs.data, 1)
            
            
            running_loss += loss.item() * images.size(0)
            correct += torch.sum(preds == labels.data)
        
        
            loss.backward()
            optimizer.step()
        
        tra_loss=running_loss/len(train_loader.dataset)
        tra_cc=100. * correct / len(train_loader.dataset)
        
        #Validate
        model.eval()
        running_loss = 0
        correct  = 0
        for i, (images, labels) in enumerate(test_loader):
            images = images.view(-1, 28*28).to(device)
            labels = labels.to(device)
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs.data, 1)
            
            # statistics
            running_loss += loss.item() * images.size(0)
            correct += torch.sum(preds == labels.data)
            
        val_loss=running_loss/len(test_loader.dataset)
        val_acc=100. * correct / len(test_loader.dataset) 
        
        
        #save results
        total_train_loss.append(tra_loss)
        total_val_loss.append(val_loss)
        total_train_acc.append(tra_cc)
        total_val_acc.append(val_acc)
        
        if epoch%2==0:
            print("Epoch: {}, train_loss: {:.4f} , val_loss: {:.4f}, tra_acc: {:.2f}, val_acc: {:.2f}".format(epoch+1, tra_loss,val_loss, tra_cc, val_acc))
       
    plt.subplot(1,2,1)
    plt.plot(total_train_acc, label="Train")
    plt.plot(total_val_acc, label="Validation")
    plt.title("Accuracy vs Iteration")
    plt.xlabel("iterations")
    plt.ylabel("Accuracy %")
    plt.tight_layout()
    plt.legend()
    
    plt.subplot(1,2,2)
    plt.plot(total_train_loss, label="Train")
    plt.plot(total_val_loss, label="Validation")
    plt.title("Loss vs Iteration")
    plt.xlabel("iterations")
    plt.ylabel("Loss")
    plt.tight_layout()
    plt.legend()
          

In [ ]:
model_early_stopping =  MLP(input_dim, hidden_dim, output_dim)

optimizer =  torch.optim.SGD(model_early_stopping .parameters(), lr=learning_rate, momentum=0.9)


In [ ]:
fit_model(model_early_stopping , optimizer, criterion, 10, device)

** Questions**
1. From the above figure does the model overfit or underfit?
2. If you were to stop training once the validation accuracy start to decrease, at what epoch (iteration) would you stop?
3. Implement the basic early stopping approche by modifying the above code.
4. Compare your results with the previous default model.

### Early stopping after T epoch, model saving  and model checkpoint
For this lab, we will implement **Early stopping after T epoch**: training for T epochs, and simply selecting the best model based on validation score over the entire T epochs by following the following steps by modifying the previous train function and call valadation(test) function after each epoch. To achieve this we will follow the follwing steps:

1. Redifine a train function and we will call it fit function.
2. Define a validation function
3. Define a model checkpoint and saving function
4. Define and call model train function

### 3.1.1 Define fit function

In [ ]:
def fit(model, optimizer, criterion, device, train_loader):
    
    
    
    running_loss = 0
    correct  = 0
    model.train()
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs.data, 1)
            
            
        running_loss += loss.item() * images.size(0)
        
        #find accuracy
        correct += torch.sum(preds == labels.data)
        
        
        loss.backward()
        optimizer.step()
        
    tra_loss=running_loss/len(train_loader.dataset)
    tra_cc=100. * correct / len(train_loader.dataset)
    
    return tra_loss, tra_cc

### 3.1.2 Define validate function.

In [ ]:
def validate(model, optimizer, criterion, device, test_loader):
    
    model.eval()
    running_loss = 0
    correct  = 0
    
    for i, (images, labels) in enumerate(test_loader):
        
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
            
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs.data, 1)
            
        # statistics
        running_loss += loss.item() * images.size(0)
        correct += torch.sum(preds == labels.data)
            
    val_loss=running_loss/len(test_loader.dataset)
    val_acc=100. * correct / len(test_loader.dataset) 
        
    return val_loss, val_acc
    

### 3.1.3 Define model save checkpoint function

This function will be used to save the best model. Pytorch offer `torch.save()` and `torch.load()` for saving and loading saved models respectively.We will use [shutill](https://docs.python.org/3/library/shutil.html) python library  to save the best model with its associated parameters. Shutill offers a number of high-level operations on files and collections of files.

In [ ]:
import shutil

def save_checkpoint(state, is_best, model_path):
    
    torch.save(state, model_path)
    
    if is_best:
        print("Saving best model")
        shutil.copyfile(model_path, 'model_best.pth.tar')



Finaly we defin a model train function

In [ ]:
def train_model(model, optimizer, criterion, device, train_loader, test_loader, num_epochs, save_path):
    
    train_loss = []
    val_loss = []
    train_acc = []
    val_acc = []
    model.to(device)
    criterion.to(device)
    best_score = 0
    
    print("Start training")
    for epoch in range(num_epochs):
        
        #fit model
        loss_tra, acc_tra = fit(model, optimizer, criterion, device, train_loader)
        train_loss.append(loss_tra)
        train_acc.append(train_acc)
        
        #validate model
        loss_val, acc_val = validate(model, optimizer, criterion, device,  test_loader)
        val_loss.append(loss_val)
        val_loss.append(acc_val)
        
        is_best = acc_val >  best_score
        best_score = max(acc_val, best_score)
        
        save_checkpoint(state={
            'epoch': epoch + 1,
            'arch': "MLP",
            'state_dict': model.state_dict(),
            'best_score': best_score,
            'optimizer' : optimizer.state_dict(),
        }, is_best=is_best,model_path=save_path)
        
        print("Best score is: {:.3f}, current score is: {:.3f}".format(best_score, acc_val))
        
        
        print("Epoch: {}, Train loss: {:.4f} , Valid loss: {:.4f},Train accuracy: {:.2f}, Valid Accuracy: {:.2f}".format(epoch, loss_tra, 
                                                                 loss_val, acc_tra, acc_val))
        print('----------------------------------------------------------------------------------------------------')
    Loss = {"train": train_loss, "val":val_loss}  
    Accuracy =  {"train": train_acc, "val":val_acc}
    
    return Loss, Accuracy   

We can run our model by calling the train_model function

In [ ]:
save_path = '../tmp/MLP_checkpoint.pth.tar'
model_early_stopping =  MLP(input_dim, hidden_dim, output_dim)
optimizer =  torch.optim.SGD(model_early_stopping .parameters(), lr=learning_rate, momentum=0.9)
loss, acc = train_model(model_early_stopping, optimizer, criterion, device,train_loader, test_loader, 10, save_path)

### Load Saved mode

To load pytorch saved modals use `torch.load`.

In [ ]:
def load_saved_modal(saved_model_path, modal, optimizer):
    if os.path.isfile(saved_model_path):
        print("=> loading checkpoint '{}'".format(saved_model_path))
        checkpoint = torch.load(saved_model_path)
        start_epoch = checkpoint['epoch']
        best_score = checkpoint['best_score']
        modal.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})".format(saved_model_path, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(saved_model_path))
        best_score = 0
        start_epoch =0    


In [ ]:
import os
load_saved_modal(save_path, model_early_stopping, optimizer)

## 3.4 Dropout 

Dropout[1] is a technique for regularizing neural networks by randomly setting some features to zero during the forward pass. 

[1] Geoffrey E. Hinton et al, "Improving neural networks by preventing co-adaptation of feature detectors", arXiv 2012

In [ ]:
def MLP_dropout(input_dim, hidden_dim, output_dim, drop_prob):
    
    model = nn.Sequential(
    nn.Linear(input_dim,hidden_dim), nn.ReLU(), nn.Dropout(p=drop_prob),
    nn.Linear(hidden_dim,hidden_dim),nn.ReLU(), nn.Dropout(p=drop_prob),
    nn.Linear(hidden_dim,output_dim)
    )
    
    return model

In [ ]:
drop_prob = 0.25
input_dim = 784 #28X28
hidden_dim = 16
output_dim = 10
model_drop = MLP_dropout(input_dim, hidden_dim, output_dim, drop_prob)
print(model_drop)

In [ ]:
optimizer =  torch.optim.SGD(model_drop.parameters(), lr=learning_rate, momentum=0.9)
total_loss_drop = train(model_drop , optimizer, criterion, 5, device)

## Task:
1. Run an experiment for different dropout probability $p=0, 0.25, 0.5, 0.75$. 
2. Compare the total loss for each dropout probability.
3. Comments on your results